In [68]:
import pandas as pd 
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 
# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드']] 
# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) 

def get_url(item_name, code_df):
    code = (code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False)).lstrip()
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
    
    print("요청 URL = {}".format(url))
    return url

item_name = '신라젠'
url = get_url(item_name, code_df)

df = pd.DataFrame()

for page in range(1, 21):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
    
df = df.dropna()

df.head()

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=215600


,날짜,종가,전일비,시가,고가,저가,거래량
1,2019.12.11,13850.0,500.0,14250.0,14350.0,13800.0,1062162.0
2,2019.12.10,14350.0,300.0,14300.0,14500.0,14100.0,1542255.0
3,2019.12.09,14050.0,300.0,13850.0,14200.0,13500.0,1594531.0
4,2019.12.06,13750.0,550.0,13300.0,13950.0,13300.0,1965387.0
5,2019.12.05,13200.0,550.0,13800.0,13850.0,13000.0,1864402.0


In [69]:
del df['날짜'], df['전일비'], df['거래량']

In [70]:
jongga_zonnal = df['종가'].shift(1, axis=0)
df = pd.concat([df, jongga_zonnal], axis=1)
df.columns = ['종가', '시가', '고가', '저가', '전일종가']
df_sub = df['종가'] - df['전일종가']
df = pd.concat([df, df_sub], axis=1)
df.rename(columns={0:"당일-전일"}, inplace = True)
df_result = df['당일-전일'] / df['전일종가']
df = pd.concat([df, df_result], axis=1)
df.rename(columns={0:"등락률"}, inplace = True)
df_new = df['등락률'] * 100
df = pd.concat([df, df_new], axis=1)
df.columns.values[7] = '등락률1'
df

,종가,시가,고가,저가,전일종가,당일-전일,등락률,등락률1
1,13850.0,14250.0,14350.0,13800.0,NaN,NaN,NaN,NaN
2,14350.0,14300.0,14500.0,14100.0,13850.0,500.0,0.036101,3.610108
3,14050.0,13850.0,14200.0,13500.0,14350.0,-300.0,-0.020906,-2.090592
4,13750.0,13300.0,13950.0,13300.0,14050.0,-300.0,-0.021352,-2.135231
5,13200.0,13800.0,13850.0,13000.0,13750.0,-550.0,-0.040000,-4.000000
...,...,...,...,...,...,...,...,...
294,74300.0,74200.0,76000.0,71300.0,78100.0,-3800.0,-0.048656,-4.865557
295,73500.0,72700.0,73800.0,72300.0,74300.0,-800.0,-0.010767,-1.076716
296,72800.0,73600.0,74400.0,72800.0,73500.0,-700.0,-0.009524,-0.952381
297,73300.0,72100.0,73900.0,71500.0,72800.0,500.0,0.006868,0.686813


In [71]:
def color_negative_red(val):
    color = 'red' if val < 0 else 'blue'
    return 'color: %s' % color

In [72]:
s = df.style.applymap(color_negative_red)
s

,종가,시가,고가,저가,전일종가,당일-전일,등락률,등락률1
1,13850,14250,14350,13800,nan,nan,nan,nan
2,14350,14300,14500,14100,13850,500,0.0361011,3.61011
3,14050,13850,14200,13500,14350,-300,-0.0209059,-2.09059
4,13750,13300,13950,13300,14050,-300,-0.0213523,-2.13523
5,13200,13800,13850,13000,13750,-550,-0.04,-4
9,13750,14050,14650,13350,13200,550,0.0416667,4.16667
10,13650,14200,14650,13550,13750,-100,-0.00727273,-0.727273
11,14400,14500,14700,14000,13650,750,0.0549451,5.49451
12,14500,15250,15600,14400,14400,100,0.00694444,0.694444
13,15150,15050,16050,15000,14500,650,0.0448276,4.48276
